<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/103_TxtSummarizerAgent_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## 🧩 Gaps vs “Best-Practice”

These are in the “Best-Practice” but intentionally omitted in your minimal pipeline—add later if/when you want a general agent loop:

1. **Tool Registry + Environment (auto DI):**
   A registry of tools and an environment that injects underscore `_dep` args and executes tools by name.&#x20;

2. **Agent loop + Capabilities:**
   A small Agent that consults a model to choose tools, with plug-in capabilities like `PlanFirstCapability` and `ProgressTrackingCapability`.&#x20;

3. **Dependencies bag (`deps`) & underscore DI pattern:**
   e.g., `def create_plan(ctx, _clock=...)` auto-injected from `ctx.deps["clock"]`.&#x20;

4. **Tool schemas / registry metadata:**
   JSON-like parameter schemas stored with each tool (useful when a planner chooses tools).&#x20;

5. **Stop conditions / guards:**
   `max_calls`, validation checks before/after tools (more relevant once you have an Agent loop).&#x20;

## 👍 Nice polish you already have

* Plan normalization (regex → clean steps).
* Prompt truncation bookkeeping (`was_truncated`, lengths).
* Configurable model + temperature via config.&#x20;

## 📌 Recommendation

* **Keep your current script as the “single-file pipeline” template.**
* When you’re ready, lift it into the **recipe’s full pattern** by adding: a tiny `ToolRegistry`, `Environment` (underscore DI), an `Agent` with `PlanFirst + ProgressTracking` capabilities, and “wiring” to connect them. You already have reference code for that in the recipe file.&#x20;

If you want, I can provide a minimal “registry + environment + agent” wrapper that runs your existing tools unchanged, so you can see the full recipe in action without rewriting your tool code.




# Centralized Progress Logging via `Environment.run`

## What changed

* **Before:** Each tool (`read_txt_file`, `save_summary`, …) called `ctx.track_progress(...)` inside the function.
* **Now:** Tools are **pure** (no logging). The **Environment** wraps every tool call and logs `started ➜ completed/error` automatically.

```python
# Old (inside tool) – removed
# ctx.track_progress("read_txt_file", "started", ...)

# New (centralized)
res = env.run("read_txt_file", file_name="foo.txt")  # Environment logs around the call
```

## Why this is better

* **Consistency by default:** Every tool call is logged the same way (no “forgot to log” bugs).
* **Cleaner tools:** Functions focus on their job (I/O, summarization) instead of plumbing.
* **Single source of truth:** One place (`Environment.run`) controls logging, retries, and error handling.
* **Easier to evolve:** To send logs to a file/DB/telemetry, update the Environment once—no tool changes.
* **Less duplication:** No double entries from both caller and tool.

## When to log inside a tool

* Only for **sub-steps** or fine-grained milestones. Prefix to distinguish:

```python
ctx.track_progress("tool:read_txt_file:chunk", "completed", "chunk=3/10")
```

## Quick checklist

* Keep `ActionContext.track_progress` (used by Environment).
* Keep logging inside `Environment.run`.
* **Remove routine logging from tools.**
* Add tool-internal logs only for special sub-steps (optional).

---

### why this makes future agents easier
**separate concerns:** tools do the work; the environment handles orchestration (logging, errors, DI).

* **faster to build:** new tools = just write the function; no plumbing to remember.
* **safer & more reliable:** every tool call is automatically logged (`started/completed/error`) in one place.
* **cleaner prototypes:** tools stay small and testable; you can run them directly or via the env.
* **easier to evolve:** want retries, timeouts, metrics, or exporting logs? add it once to the env; all tools benefit.
* **consistent behavior:** uniform progress logs and error handling across the whole agent.

### sticky note for your notebook

> “Keep tools pure. Let `Environment.run` handle logging & orchestration. This reduces bugs, speeds iteration, and scales cleanly.”



In [2]:
!pip -q install openai python-dotenv

from openai import OpenAI
from dotenv import load_dotenv
import os
import textwrap
import time
import re
from dataclasses import dataclass
import inspect
from typing import Callable

# ---- Setup ----
load_dotenv('/content/API_KEYS.env')
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("OPENAI_API_KEY not found in /content/API_KEYS.env")
client = OpenAI(api_key=api_key)


# ---------------- Memory + Context ----------------
class ScratchMemory:
    def __init__(self):
        self.store = {}

    def get(self, key, default=None):   # <-- add default
        return self.store.get(key, default)

    def set(self, key, value):
        self.store[key] = value

VALID_STATUSES = {"started", "completed", "error"}

class ActionContext:
    def __init__(self, memory, llm, config=None, deps=None):
        self.memory = memory
        self.llm = llm
        self.config = config or {}
        self.deps = deps or {}

    # --- progress helpers ---
    def track_progress(self, step, status, note=""):
        if status not in VALID_STATUSES:
            raise ValueError(f"Invalid status '{status}'. Use {VALID_STATUSES}.")
        log = self.memory.get("progress_log") or []
        log.append({
            "step": step,
            "status": status,
            "note": note,
            "time": time.strftime("%Y-%m-%d %H:%M:%S"),
        })
        self.memory.set("progress_log", log)

    def print_progress(self):
        log = self.memory.get("progress_log") or []
        print("\n📊 Progress Log:")
        for e in log:
            t = f" ({e.get('time')})" if e.get("time") else ""
            note = f" — {e['note']}" if e.get("note") else ""
            print(f"- [{e['status']}] {e['step']}{t}{note}")

    def last_completed_step(self):
        log = self.memory.get("progress_log") or []
        for e in reversed(log):
            if e.get("status") == "completed":
                return e.get("step")
        return None

    def first_error(self):
        log = self.memory.get("progress_log") or []
        for e in log:
            if e.get("status") == "error":
                return e
        return None

# ---------------- LLM Wrapper ----------------
class OpenAILLM:
    def __init__(self, client, model="gpt-4o-mini", temperature=0.2):
        self.client = client
        self.model = model
        self.temperature = temperature

    def complete(self, prompt, **kwargs):
        temp = kwargs.get("temperature", self.temperature)
        resp = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=temp,
        )
        return resp.choices[0].message.content

# ---------------- Tool: create_plan ----------------
def create_plan(ctx):
    goal = ctx.memory.get("goal")
    if not goal:
        return {"error": "No goal provided in memory."}

    prompt = f"""You are an expert task planner. Given the goal below, break it down into a clear, short list of steps.

Goal: {goal}

Respond ONLY with a numbered list, one step per line. No extra prose."""
    raw = ctx.llm.complete(prompt).strip()

    # Prefer numbered steps like "1. ...", "2) ..."
    numbered = re.findall(r'^\s*(?:\d+[\).\s-]+)\s*(.+)$', raw, flags=re.M)

    if numbered:
        steps = numbered
    else:
        # Fallback to bullets like "- ...", "* ...", "• ..."
        bullets = re.findall(r'^\s*(?:[-*•]\s+)(.+)$', raw, flags=re.M)
        steps = bullets if bullets else [ln.strip() for ln in raw.splitlines() if ln.strip()]

    # Normalize: collapse spaces, trim punctuation, drop empties/dupes
    norm = []
    seen = set()
    for s in steps:
        s = re.sub(r'\s+', ' ', s).strip(' .')
        if s and s.lower() not in seen:
            seen.add(s.lower())
            norm.append(s)

    ctx.memory.set("plan", norm)
    return {"message": "Plan created from goal.", "steps": norm}

# ---------------- Tool: read_txt_file ----------------
def read_txt_file(ctx, file_name):
    folder = ctx.config.get("input_folder")
    path = os.path.join(folder, file_name)

    if not os.path.exists(path):
        return {"error": f"File not found: {path}"}

    with open(path, "r", encoding="utf-8") as f:
        text = f.read()

    ctx.memory.set("file_name", file_name)
    ctx.memory.set("raw_text", text)
    return {"message": "File read successfully.", "length": len(text)}


# ---------------- Tool: generate_summary_prompt ----------------
def generate_summary_prompt(ctx, max_len=2000):
    text = ctx.memory.get("raw_text")
    if not text:
        return {"error": "No raw text found in memory."}

    # Truncate + record bookkeeping
    truncated = len(text) > max_len
    short_text = text[:max_len]
    ctx.memory.set("was_truncated", truncated)
    ctx.memory.set("source_length", len(text))
    ctx.memory.set("used_length", len(short_text))

    prompt = f"""You are an expert technical writer.

Summarize the following content into a set of clear, concise bullet points. Focus on the main ideas, and skip boilerplate or excessive detail.

Text:
\"\"\"
{short_text}
\"\"\"

Summary:"""

    ctx.memory.set("summary_prompt", prompt)
    return {
        "message": "Summary prompt created.",
        "prompt_preview": prompt[:600],
        "truncated": truncated,
        "used": len(short_text),
        "total": len(text),
    }

# ---------------- Tool: summarize ----------------
def summarize(ctx):
    prompt = ctx.memory.get("summary_prompt")
    if not prompt:
        return {"error": "No summary prompt found in memory."}
    try:
        response = ctx.llm.complete(prompt)
    except Exception as e:
        return {"error": f"LLM error: {e}"}

    ctx.memory.set("summary", response)
    return {"message": "Summary completed.", "summary_preview": response[:1000]}

# ---------------- Tool: save_summary ----------------
def save_summary(ctx, out_name=None):
    summary = ctx.memory.get("summary")
    if not summary:
        return {"error": "No summary in memory."}

    out_dir = ctx.config.get("output_folder")
    if not out_dir:
        return {"error": "No output_folder in config."}

    os.makedirs(out_dir, exist_ok=True)
    base = out_name
    if not base:
        src = ctx.memory.get("file_name", "summary")
        root, _ = os.path.splitext(os.path.basename(src))
        base = f"{root}_summary.txt"

    path = os.path.join(out_dir, base)
    with open(path, "w", encoding="utf-8") as f:
        f.write(summary)

    ctx.memory.set("summary_path", path)
    return {"message": "Summary saved.", "path": path}

# -----------------Tool Registry ----------------
@dataclass
class ToolDef:
    name: str
    func: Callable
    description: str = ""
    schema: dict | None = None  # optional metadata hook for later

class ToolRegistry:
    def __init__(self):
        self._tools = {}

    def register(self, tool: ToolDef):
        self._tools[tool.name] = tool

    def get(self, name: str) -> ToolDef:
        if name not in self._tools:
            raise KeyError(f"Unknown tool: {name}")
        return self._tools[name]

    def list(self):
        return list(self._tools.keys())

#------------Environment--------------
class Environment:
    def __init__(self, ctx: ActionContext, registry: ToolRegistry):
        self.ctx = ctx
        self.registry = registry

    def run(self, tool_name: str, **kwargs):
        tool = self.registry.get(tool_name)
        fn = tool.func
        sig = inspect.signature(fn)

        # --- build call args with auto-DI ---
        call_args = {}
        for pname, param in sig.parameters.items():
            if pname == "ctx":
                call_args["ctx"] = self.ctx
            elif pname.startswith("_"):  # underscore dep, e.g. _clock
                dep_name = pname[1:]
                if dep_name not in self.ctx.deps:
                    raise KeyError(f"Missing dep '{dep_name}' for tool '{tool_name}'")
                call_args[pname] = self.ctx.deps[dep_name]
            else:
                if pname in kwargs:
                    call_args[pname] = kwargs[pname]
                elif param.default is not inspect._empty:
                    # has default; okay to omit
                    pass
                else:
                    raise TypeError(f"Missing required arg '{pname}' for tool '{tool_name}'")

        # --- progress logging wrapper ---
        self.ctx.track_progress(tool_name, "started", note=str({k:v for k,v in kwargs.items()}))
        try:
            result = fn(**call_args)
        except Exception as e:
            self.ctx.track_progress(tool_name, "error", note=str(e))
            raise
        else:
            status_note = ""
            if isinstance(result, dict) and "error" in result:
                self.ctx.track_progress(tool_name, "error", note=str(result["error"])[:180])
                return result
            if isinstance(result, dict):
                status_note = result.get("message", "")[:120]
            self.ctx.track_progress(tool_name, "completed", note=status_note)
            return result


# ------------build registry------------
registry = ToolRegistry()
registry.register(ToolDef("create_plan", create_plan, "Create a plan from goal"))
registry.register(ToolDef("read_txt_file", read_txt_file, "Read a .txt file"))
registry.register(ToolDef("generate_summary_prompt", generate_summary_prompt, "Construct summarization prompt"))
registry.register(ToolDef("summarize", summarize, "Run LLM summarization"))
registry.register(ToolDef("save_summary", save_summary, "Persist summary to disk"))

# ---------------- Setup + Test ----------------
# Set up memory and config
memory = ScratchMemory()
memory.set("goal", "Summarize the content of a text file.")
config = {
    "input_folder": "/content/files",
    "output_folder": "/content/output"
}

# Set up LLM and context (inject config)
llm = OpenAILLM(
    client,
    model=config.get("model", "gpt-4o-mini"),
    temperature=config.get("temperature", 0.2),
)

#-----------Set up ActionContext -------------
ctx = ActionContext(memory=memory, llm=llm, config=config)

os.makedirs(ctx.config["input_folder"], exist_ok=True)
os.makedirs(ctx.config["output_folder"], exist_ok=True)
ctx.track_progress("setup", "completed", "goal + config injected")

#----------Make/Run Environment----------#
env = Environment(ctx, registry)

# 1) plan
res = env.run("create_plan")
print(res["message"]); print("\nPlan:")
for s in res["steps"]:
    print("-", s)

# 2) read file
file_name = "004_AGENT_Tools.txt"
res = env.run("read_txt_file", file_name=file_name)
print("✅", res["message"], "chars:", res["length"])

# optional preview (no second read)
print("\n📄 File Preview:\n")
raw_text = ctx.memory.get("raw_text") or ""
print(textwrap.fill(raw_text[:600], width=80, subsequent_indent="  "))

# 3) prompt
res = env.run("generate_summary_prompt")
if res.get("truncated"):
    print(f"(note) Prompt truncated to {res['used']} / {res['total']} chars.")
print("\n🧾 Prompt Preview:\n")
print(textwrap.fill(res["prompt_preview"], width=80, subsequent_indent="  "))

# 4) summarize
res = env.run("summarize")
print("\n✅", res["message"])
print("\n📝 Summary Preview:\n")
print(textwrap.fill(res["summary_preview"], width=80, subsequent_indent="  "))

# 5) save
res = env.run("save_summary")
print(res.get("message", res.get("error")))
if "path" in res:
    print("📄 Saved to:", res["path"])

# snapshot/log
print("\n" + "="*80)
print("📦 ActionContext Snapshot")
ctx.print_progress()


Plan created from goal.

Plan:
- Open the text file using a text editor or programming language
- Read the entire content of the file
- Identify the main ideas and key points in the text
- Take notes on important details and supporting information
- Organize the notes into a coherent structure
- Write a concise summary based on the organized notes
- Review and edit the summary for clarity and accuracy
- Save or share the summary as needed
✅ File read successfully. chars: 3107

📄 File Preview:

 #===========AI-Agent Tool Descriptions and Naming  Describing Tools to the
  Agent  When developing an agentic AI system, one of the most critical aspects
  is ensuring that the agent understands the tools it has access to. In our
  previous tutorial, we explored how an AI agent interacts with an environment.
  Now, we extend that discussion to focus on tool definition, particularly the
  importance of naming, parameters, and structured metadata.  Example:
  Automating Documentation for Python C

Here’s a quick “feature check” against the best-practice list, plus tiny, surgical next steps.

### 1) Tool Registry + Environment (auto-DI)

**Status:** ✅ Implemented.
You have a registry, and `Environment.run/execute` wraps every call and handles logging & arg injection. Nice.&#x20;

**If you want to polish:** already solid. Keep using env-only logging for consistency.

---

### 2) Agent loop + Capabilities

**Status:** ❌ Not yet in the summarizer pipeline (it’s a linear script).
Best-practice shows a tiny Agent that decides which tool to call next and lets you plug capabilities like `PlanFirstCapability` / `ProgressTrackingCapability`.&#x20;

**Smallest next step (no rewrites to your tools):**

```python
class ScriptedAgent:
    def __init__(self, env, steps):
        self.env = env
        self.steps = steps  # e.g., [("create_plan", {}), ("read_txt_file", {"file_name": ...}), ...]
    def run(self):
        for name, kwargs in self.steps:
            res = self.env.run(name, **kwargs)
            if isinstance(res, dict) and "error" in res:
                return {"final": f"stopped at {name}: {res['error']}"}
        return {"final": "done"}
```

Then build steps from your current sequence and call `agent.run()`. Later, swap in a planner model and add capabilities.

---

### 3) Dependencies bag (`deps`) & underscore DI

**Status:** ✅ `ctx.deps` exists; **Partial** usage.
Best practice encourages underscore params (e.g., `_clock`, `_fs`) that are auto-filled by the env.&#x20;

**Tiny adoption example (optional):**

```python
def save_summary(ctx, out_name=None, _fs=os):
    # use _fs.path.join, _fs.makedirs, etc.
```

Now you can inject a fake FS in tests: `ctx.deps["fs"] = FakeFS`.

---

### 4) Tool schemas / registry metadata

**Status:** ⚠️ Usually missing/empty in the summarizer registry.
Recipe suggests JSON-like parameter schemas per tool so planners (or humans) know what to pass.&#x20;

**Minimal add (one line per registration):**

```python
registry.register(ToolDef(
  "read_txt_file", read_txt_file, "Read a .txt file",
  schema={"type":"object","properties":{"file_name":{"type":"string"}}, "required":["file_name"]}
))
registry.register(ToolDef(
  "generate_summary_prompt", generate_summary_prompt, "Build prompt",
  schema={"type":"object","properties":{"max_len":{"type":"integer"}}, "required":[]}
))
registry.register(ToolDef(
  "summarize", summarize, "LLM summarization",
  schema={"type":"object","properties":{},"required":[]}
))
registry.register(ToolDef(
  "save_summary", save_summary, "Persist summary",
  schema={"type":"object","properties":{"out_name":{"type":"string"}}, "required":[]}
))
```

---

### 5) Stop conditions / guards

**Status:** ❌ (not needed for a straight script, but essential once you add an Agent loop).
Best practice: `max_calls`, simple validation before/after tool calls, and uniform error envelopes.&#x20;

**Lightweight guard right now (2 lines in your env):**

```python
# After calling a tool:
if isinstance(result, dict) and "error" in result:
    self.ctx.track_progress(tool_name, "error", note=str(result["error"])[:180])
    return result
self.ctx.track_progress(tool_name, "completed", note=result.get("message","")[:120] if isinstance(result, dict) else "")
```

When you add an Agent class, include `max_calls=5–8` and bail out cleanly with a final message.

---

## TL;DR

* **Already have:** Registry + Environment ✅, centralized logging ✅, DI bag ✅.
* **Add next (smallest steps):** (a) scripted mini-Agent loop, (b) schemas on registry entries, (c) underscore DI on 1–2 tools, (d) treat `{"error":...}` as error in env, and (e) later, `max_calls` once you flip to a planning Agent.&#x20;





## Why add a schema to each tool?

**What it is:** a tiny JSON-like metadata block that describes a tool’s inputs (names, types, which are required).

**Why it helps (even before you have a planner):**

* **Self-documenting tools:** Anyone (or future-you) can see how to call a tool without reading its code.
* **Pre-run validation:** Catch “missing `file_name`” or wrong types *before* the tool runs.
* **Future-ready for planning:** If/when you ask an LLM to choose and call tools, you can hand it the schemas so it knows the parameter names and shapes.
* **Auto-UI / CLI forms (later):** Schemas can be used to auto-generate simple UIs or arg parsers.

**Minimal example you already started using:**

```python
registry.register(ToolDef(
  "read_txt_file", read_txt_file, "Read a .txt file",
  schema={
    "type": "object",
    "properties": {"file_name": {"type": "string"}},
    "required": ["file_name"]
  }
))
```

**(Optional) Light validation before running:**

```python
def _validate(schema, kwargs):
    if not schema: return None
    missing = [k for k in schema.get("required", []) if k not in kwargs]
    if missing: return f"Missing required: {missing}"
    types = {"string": str, "integer": int, "number": (int, float), "boolean": bool}
    for k, spec in (schema.get("properties") or {}).items():
        if k in kwargs and "type" in spec:
            if not isinstance(kwargs[k], types[spec["type"]]):
                return f"Bad type for '{k}': expected {spec['type']}"
    return None
```

You could call `_validate(tool.schema, kwargs)` inside `Environment.run` and, if it returns a message, log an error and return `{"error": ...}`. Totally optional but handy.





## What does the `ScriptedAgent` do, and why add it?

**What it is:** a *tiny orchestrator* that runs a list of steps, e.g.

```python
steps = [
  ("create_plan", {}),
  ("read_txt_file", {"file_name": "004_AGENT_Tools.txt"}),
  ("generate_summary_prompt", {}),
  ("summarize", {}),
  ("save_summary", {})
]
agent = ScriptedAgent(env, steps)
agent.run(max_calls=10)
```

**How it differs from `Environment`:**

* **Environment** = “call one tool safely” (auto-DI, logging, error handling).
* **ScriptedAgent** = “run a whole pipeline of tools” (a loop over steps, with guards like `max_calls` and `stop_on_error`).

**Why it helps:**

* **Separation of concerns:** tools do work; env handles a single call; agent sequences many calls.
* **One place for run policy:** stop on first error, cap total calls, later add retries/ backoff—all in the agent.
* **Swappable brains:** today your steps are a fixed list; tomorrow an LLM (a planner) can emit the steps array using your tool schemas—no tool code changes.
* **Capability hooks (later):** easy to add small mix-ins like `PlanFirst` (prepend `create_plan`) or `ProgressTracking` without touching tools.

**TL;DR**

* **Schema** = machine-readable “how to call this tool” → validation, docs, and future planning.
* **ScriptedAgent** = the simplest agent loop → turns your linear notebook into a reusable, policy-controlled runner that you can later make model-driven.


In [4]:
!pip -q install openai python-dotenv

from openai import OpenAI
from dotenv import load_dotenv
import os
import textwrap
import time
import re
import inspect
from typing import Callable, Optional
from dataclasses import dataclass

# ---- Setup ----
load_dotenv('/content/API_KEYS.env')
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("OPENAI_API_KEY not found in /content/API_KEYS.env")
client = OpenAI(api_key=api_key)


# ---------------- Memory + Context ----------------
class ScratchMemory:
    def __init__(self):
        self.store = {}

    def get(self, key, default=None):   # <-- add default
        return self.store.get(key, default)

    def set(self, key, value):
        self.store[key] = value

VALID_STATUSES = {"started", "completed", "error"}

class ActionContext:
    def __init__(self, memory, llm, config=None, deps=None):
        self.memory = memory
        self.llm = llm
        self.config = config or {}
        self.deps = deps or {}

    # --- progress helpers ---
    def track_progress(self, step, status, note=""):
        if status not in VALID_STATUSES:
            raise ValueError(f"Invalid status '{status}'. Use {VALID_STATUSES}.")
        log = self.memory.get("progress_log") or []
        log.append({
            "step": step,
            "status": status,
            "note": note,
            "time": time.strftime("%Y-%m-%d %H:%M:%S"),
        })
        self.memory.set("progress_log", log)

    def print_progress(self):
        log = self.memory.get("progress_log") or []
        print("\n📊 Progress Log:")
        for e in log:
            t = f" ({e.get('time')})" if e.get("time") else ""
            note = f" — {e['note']}" if e.get("note") else ""
            print(f"- [{e['status']}] {e['step']}{t}{note}")

    def last_completed_step(self):
        log = self.memory.get("progress_log") or []
        for e in reversed(log):
            if e.get("status") == "completed":
                return e.get("step")
        return None

    def first_error(self):
        log = self.memory.get("progress_log") or []
        for e in log:
            if e.get("status") == "error":
                return e
        return None

# ---------------- LLM Wrapper ----------------
class OpenAILLM:
    def __init__(self, client, model="gpt-4o-mini", temperature=0.2):
        self.client = client
        self.model = model
        self.temperature = temperature

    def complete(self, prompt, **kwargs):
        temp = kwargs.get("temperature", self.temperature)
        resp = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=temp,
        )
        return resp.choices[0].message.content

# ---------------- Tool: create_plan ----------------
def create_plan(ctx):
    goal = ctx.memory.get("goal")
    if not goal:
        return {"error": "No goal provided in memory."}

    prompt = f"""You are an expert task planner. Given the goal below, break it down into a clear, short list of steps.

Goal: {goal}

Respond ONLY with a numbered list, one step per line. No extra prose."""
    raw = ctx.llm.complete(prompt).strip()

    # Prefer numbered steps like "1. ...", "2) ..."
    numbered = re.findall(r'^\s*(?:\d+[\).\s-]+)\s*(.+)$', raw, flags=re.M)

    if numbered:
        steps = numbered
    else:
        # Fallback to bullets like "- ...", "* ...", "• ..."
        bullets = re.findall(r'^\s*(?:[-*•]\s+)(.+)$', raw, flags=re.M)
        steps = bullets if bullets else [ln.strip() for ln in raw.splitlines() if ln.strip()]

    # Normalize: collapse spaces, trim punctuation, drop empties/dupes
    norm = []
    seen = set()
    for s in steps:
        s = re.sub(r'\s+', ' ', s).strip(' .')
        if s and s.lower() not in seen:
            seen.add(s.lower())
            norm.append(s)

    ctx.memory.set("plan", norm)
    return {"message": "Plan created from goal.", "steps": norm}

# ---------------- Tool: read_txt_file ----------------
def read_txt_file(ctx, file_name):
    folder = ctx.config.get("input_folder")
    path = os.path.join(folder, file_name)

    if not os.path.exists(path):
        return {"error": f"File not found: {path}"}

    with open(path, "r", encoding="utf-8") as f:
        text = f.read()

    ctx.memory.set("file_name", file_name)
    ctx.memory.set("raw_text", text)
    return {"message": "File read successfully.", "length": len(text)}


# ---------------- Tool: generate_summary_prompt ----------------
def generate_summary_prompt(ctx, max_len=2000):
    text = ctx.memory.get("raw_text")
    if not text:
        return {"error": "No raw text found in memory."}

    # Truncate + record bookkeeping
    truncated = len(text) > max_len
    short_text = text[:max_len]
    ctx.memory.set("was_truncated", truncated)
    ctx.memory.set("source_length", len(text))
    ctx.memory.set("used_length", len(short_text))

    prompt = f"""You are an expert technical writer.

Summarize the following content into a set of clear, concise bullet points. Focus on the main ideas, and skip boilerplate or excessive detail.

Text:
\"\"\"
{short_text}
\"\"\"

Summary:"""

    ctx.memory.set("summary_prompt", prompt)
    return {
        "message": "Summary prompt created.",
        "prompt_preview": prompt[:600],
        "truncated": truncated,
        "used": len(short_text),
        "total": len(text),
    }

# ---------------- Tool: summarize ----------------
def summarize(ctx):
    prompt = ctx.memory.get("summary_prompt")
    if not prompt:
        return {"error": "No summary prompt found in memory."}
    try:
        response = ctx.llm.complete(prompt)
    except Exception as e:
        return {"error": f"LLM error: {e}"}

    ctx.memory.set("summary", response)
    return {"message": "Summary completed.", "summary_preview": response[:1000]}

# ---------------- Tool: save_summary ----------------
def save_summary(ctx, out_name=None):
    summary = ctx.memory.get("summary")
    if not summary:
        return {"error": "No summary in memory."}

    out_dir = ctx.config.get("output_folder")
    if not out_dir:
        return {"error": "No output_folder in config."}

    os.makedirs(out_dir, exist_ok=True)
    base = out_name
    if not base:
        src = ctx.memory.get("file_name", "summary")
        root, _ = os.path.splitext(os.path.basename(src))
        base = f"{root}_summary.txt"

    path = os.path.join(out_dir, base)
    with open(path, "w", encoding="utf-8") as f:
        f.write(summary)

    ctx.memory.set("summary_path", path)
    return {"message": "Summary saved.", "path": path}

# -----------------Tool Registry ----------------
@dataclass
class ToolDef:
    name: str
    func: Callable
    description: str = ""
    schema: dict | None = None
    returns: dict | None = None   # <-- add this line

class ToolRegistry:
    def __init__(self):
        self._tools = {}

    def register(self, tool: ToolDef):
        self._tools[tool.name] = tool

    def get(self, name: str) -> ToolDef:
        if name not in self._tools:
            raise KeyError(f"Unknown tool: {name}")
        return self._tools[name]

    def list(self):
        return list(self._tools.keys())

# ------------build registry------------
registry = ToolRegistry()

registry.register(ToolDef(
    "create_plan",
    create_plan,
    "Create a plan from goal",
    schema={ "type": "object", "properties": {}, "required": [] },   # no kwargs
    returns={
        "type": "object",
        "properties": {
            "message": { "type": "string" },
            "steps":   { "type": "array", "items": { "type": "string" } }
        },
        "required": ["message", "steps"]
    }
))

registry.register(ToolDef(
  "read_txt_file", read_txt_file, "Read a .txt file from input_folder",
  schema={
    "type": "object",
    "properties": {"file_name": {"type": "string"}},
    "required": ["file_name"]
  },
  returns={
    "type": "object",
    "properties": {"message": {"type": "string"}, "length": {"type": "integer"}},
    "required": ["message"]
  },
  # side_effects=("read",), version="1.0"
))

registry.register(ToolDef(
  "generate_summary_prompt", generate_summary_prompt, "Build a summarization prompt",
  schema={
    "type": "object",
    "properties": {"max_len": {"type": "integer", "minimum": 1}},
    "required": []
  }
))

registry.register(ToolDef(
  "summarize", summarize, "Run LLM summarization",
  schema={"type": "object", "properties": {}, "required": []}
))

registry.register(ToolDef(
  "save_summary", save_summary, "Persist summary to output_folder",
  schema={
    "type": "object",
    "properties": {"out_name": {"type": "string"}},
    "required": []
  },
  # side_effects=("write",)
))

def _validate(schema, kwargs):
    if not schema:
        return None
    missing = [k for k in schema.get("required", []) if k not in kwargs]
    if missing:
        return f"Missing required: {missing}"
    types = {"string": str, "integer": int, "number": (int, float), "boolean": bool}
    for k, spec in (schema.get("properties") or {}).items():
        if k in kwargs and "type" in spec:
            py_t = types.get(spec["type"])
            if py_t and not isinstance(kwargs[k], py_t):
                return f"Bad type for '{k}': expected {spec['type']}"
    return None


#------------Environment--------------
class Environment:
    def __init__(self, ctx: ActionContext, registry: ToolRegistry):
        self.ctx = ctx
        self.registry = registry

    def run(self, tool_name: str, **kwargs):
        tool = self.registry.get(tool_name)
        fn = tool.func
        sig = inspect.signature(fn)

        # --- schema validation BEFORE starting/logging/calling ---
        err = _validate(tool.schema, kwargs)
        if err:
            self.ctx.track_progress(tool.name, "error", note=err[:180])
            return {"error": err}

        # --- build call args with auto-DI ---
        call_args = {}
        for pname, param in sig.parameters.items():
            if pname == "ctx":
                call_args["ctx"] = self.ctx
            elif pname.startswith("_"):  # underscore dep, e.g. _clock
                dep_name = pname[1:]
                if dep_name not in self.ctx.deps:
                    raise KeyError(f"Missing dep '{dep_name}' for tool '{tool_name}'")
                call_args[pname] = self.ctx.deps[dep_name]
            else:
                if pname in kwargs:
                    call_args[pname] = kwargs[pname]
                elif param.default is not inspect._empty:
                    pass  # ok, has default
                else:
                    raise TypeError(f"Missing required arg '{pname}' for tool '{tool_name}'")

        # --- progress logging wrapper ---
        self.ctx.track_progress(tool.name, "started", note=str(kwargs))
        try:
            result = fn(**call_args)
        except Exception as e:
            self.ctx.track_progress(tool.name, "error", note=str(e))
            raise
        else:
            # treat {"error": "..."} as an error outcome
            if isinstance(result, dict) and "error" in result:
                self.ctx.track_progress(tool.name, "error", note=str(result["error"])[:180])
                return result

            status_note = result.get("message", "")[:120] if isinstance(result, dict) else ""
            self.ctx.track_progress(tool.name, "completed", note=status_note)
            return result

# ---------- Scripted Agent ----------
class ScriptedAgent:
    def __init__(self, env, steps):
        self.env = env
        self.steps = steps  # [("tool_name", {"arg": ...}), ...]

    def run(self, max_calls=None, stop_on_error=True):
        calls = 0
        for name, kwargs in self.steps:
            if max_calls is not None and calls >= max_calls:
                return {"final": f"stopped: max_calls={max_calls}"}
            res = self.env.run(name, **(kwargs or {}))
            calls += 1
            if stop_on_error and isinstance(res, dict) and "error" in res:
                return {"final": f"stopped at {name}: {res['error']}"}
        return {"final": "done"}


# ---------------- Setup + Test ----------------
# Set up memory and config
memory = ScratchMemory()
memory.set("goal", "Summarize the content of a text file.")
config = {
    "input_folder": "/content/files",
    "output_folder": "/content/output"
}

# Set up LLM and context (inject config)
llm = OpenAILLM(
    client,
    model=config.get("model", "gpt-4o-mini"),
    temperature=config.get("temperature", 0.2),
)

#-----------Set up ActionContext -------------
ctx = ActionContext(memory=memory, llm=llm, config=config)

os.makedirs(ctx.config["input_folder"], exist_ok=True)
os.makedirs(ctx.config["output_folder"], exist_ok=True)
ctx.track_progress("setup", "completed", "goal + config injected")

#----------Make/Run Environment----------#
env = Environment(ctx, registry)

# --- define the steps your agent should run ---
file_name = "004_AGENT_Tools.txt"
steps = [
    ("create_plan", {}),
    ("read_txt_file", {"file_name": file_name}),
    ("generate_summary_prompt", {}),  # or {"max_len": 2400}
    ("summarize", {}),
    ("save_summary", {}),
]

# --- run the agent ---
agent = ScriptedAgent(env, steps)
final = agent.run(max_calls=10)  # optional guard
print("Agent result:", final["final"])

# --- pretty prints from memory (no second calls) ---
plan = ctx.memory.get("plan") or []
print("\nPlan:")
for s in plan:
    print("-", s)

raw_text = ctx.memory.get("raw_text") or ""
print("\n📄 File Preview:\n")
print(textwrap.fill(raw_text[:600], width=80, subsequent_indent="  "))

prompt = ctx.memory.get("summary_prompt") or ""
print("\n🧾 Prompt Preview:\n")
print(textwrap.fill(prompt[:600], width=80, subsequent_indent="  "))

summary = ctx.memory.get("summary") or ""
print("\n📝 Summary Preview:\n")
print(textwrap.fill(summary[:1000], width=80, subsequent_indent="  "))

if ctx.memory.get("summary_path"):
    print("\n📄 Saved to:", ctx.memory.get("summary_path"))

print("\n" + "="*80)
print("📦 ActionContext Snapshot")
ctx.print_progress()



Agent result: done

Plan:
- Open the text file using a text editor or programming tool
- Read the entire content of the file
- Identify the main ideas and key points in the text
- Take notes on important details and supporting information
- Organize the notes into a coherent structure (e.g., introduction, body, conclusion)
- Write a concise summary based on the organized notes
- Review and edit the summary for clarity and brevity
- Save or export the summary as needed

📄 File Preview:

 #===========AI-Agent Tool Descriptions and Naming  Describing Tools to the
  Agent  When developing an agentic AI system, one of the most critical aspects
  is ensuring that the agent understands the tools it has access to. In our
  previous tutorial, we explored how an AI agent interacts with an environment.
  Now, we extend that discussion to focus on tool definition, particularly the
  importance of naming, parameters, and structured metadata.  Example:
  Automating Documentation for Python Code Imag

Here’s a tight audit of your **summarizer agent** against the recipe, plus the smallest patches (if you want 1:1 alignment).&#x20;

## Recipe → Your implementation

| Recipe component         | What the recipe expects                               | Status in your agent | Notes / tiny patch if you want perfect alignment                                                                                                                         |
| ------------------------ | ----------------------------------------------------- | -------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------ |
| **Tool**                 | Small, stateless functions; use ctx + DI              | ✅                    | Your tools are pure and use `ctx` only. Nice.                                                                                                                            |
| **Tool Registry**        | Central list with metadata (schema)                   | ✅ (with schemas)     | You added schemas; consider adding optional `returns` metadata (you already added the `returns` field in `ToolDef`).                                                     |
| **ActionContext**        | Memory, config, deps bag                              | ✅                    | You’re using `memory/config/deps` cleanly.                                                                                                                               |
| **Environment**          | Executes tools, auto-injects underscore `_deps`, logs | ✅                    | You centralized progress logging and added schema validation before calls—exactly what the recipe suggests.                                                              |
| **Agent**                | Loop that chooses tools                               | ✅ (ScriptedAgent)    | Your `ScriptedAgent` is a simple, explicit sequence. That satisfies the “Agent” slot; later you can swap in a planner.                                                   |
| **Capabilities**         | Modular hooks (PlanFirst, ProgressTracking)           | ➖ (not wired)        | You don’t **need** them for a scripted run. If you want parity, add no-op hooks or a tiny `PlanFirst` that asserts the first step is `create_plan`.                      |
| **Dependencies**         | DI via underscore params                              | ➖/Partial            | Add a small example (e.g., `save_summary(ctx, out_name=None, _fs=os)`) so tests can inject a fake FS: `ctx.deps["fs"]=FakeFS`.                                           |
| **Wiring**               | Clear, minimal                                        | ✅                    | You register tools → build ctx → env → agent → run. ✔️                                                                                                                   |
| **PlanFirst habit**      | Always start with `create_plan`                       | ✅                    | First step in your `steps` list is `create_plan`.                                                                                                                        |
| **Progress logging**     | Append small entries; readable prints                 | ✅                    | Centralized in `Environment.run`. You also print a clean snapshot at the end.                                                                                            |
| **Stop conditions**      | `max_calls`, guard rails                              | ✅                    | `ScriptedAgent.run(max_calls=...)` already present.                                                                                                                      |
| **Schema checks**        | Validate before/after tool calls                      | ✅ (pre-call)         | You validate before executing; treating `{"error": ...}` as error is in place.                                                                                           |
| **Memory naming**        | `current_plan` used in notes                          | ➖                    | You store `"plan"`. Optional: alias to `"current_plan"` for literal recipe wording.                                                                                      |
| **track\_progress tool** | Domain tool variant (besides env logging)             | ➖                    | Not necessary (env logging covers it). If you want parity with the recipe’s demo, add a tiny `track_progress` tool that appends to `memory["progress"]` and register it. |

## Minimal “parity patches” (optional)

1. **Underscore DI example** (future-friendly testing)

```python
def save_summary(ctx, out_name=None, _fs=os):
    summary = ctx.memory.get("summary")
    if not summary:
        return {"error": "No summary in memory."}
    out_dir = ctx.config.get("output_folder")
    if not out_dir:
        return {"error": "No output_folder in config."}
    _fs.makedirs(out_dir, exist_ok=True)
    src = ctx.memory.get("file_name", "summary")
    base = out_name or f"{os.path.splitext(os.path.basename(src))[0]}_summary.txt"
    path = _fs.path.join(out_dir, base)
    with _fs.open(path, "w", encoding="utf-8") as f:
        f.write(summary)
    ctx.memory.set("summary_path", path)
    return {"message": "Summary saved.", "path": path}
```

Then you can test with `ctx.deps["fs"] = FakeFS()`.

2. **Capability stubs** (only if you want the hook points now)

```python
class PlanFirstCapability:
    def on_before_run(self, steps):
        assert steps and steps[0][0] == "create_plan", "PlanFirst required."

# use it like:
caps = [PlanFirstCapability()]
for cap in caps:
    if hasattr(cap, "on_before_run"): cap.on_before_run(steps)
agent = ScriptedAgent(env, steps)
```

(Your ScriptedAgent doesn’t use capabilities today—that’s fine; this is just to match the recipe’s structure.)

3. **Alias memory key (optional cosmetic)**

```python
plan = ctx.memory.get("plan") or []
ctx.memory.set("current_plan", plan)
```

## Verdict

* For a **summarizer** agent, you’ve implemented the **core best practices**: registry, env with DI + validation, centralized logging, and a small agent loop with stop conditions.
* What remains are **optional** polish items (underscore DI demo in one tool; a capability stub; a `current_plan` alias) that make your build line up *exactly* with the recipe’s terminology and expansion path.&#x20;


